In [1]:
import numpy as np
import pandas as pd
import kaggle
import pymysql

In [2]:
kaggle.api.authenticate()

In [5]:
kaggle.api.dataset_download_files(dataset = "shashwatwork/dataco-smart-supply-chain-for-big-data-analysis", path=".", unzip=True)

Dataset URL: https://www.kaggle.com/datasets/shashwatwork/dataco-smart-supply-chain-for-big-data-analysis


In [6]:
import os
print(os.listdir("."))

['DataCoSupplyChainDataset.csv', 'DescriptionDataCoSupplyChain.csv', 'Supply_Chain_Analysis.ipynb', 'tokenized_access_logs.csv']


In [17]:
file_path = r"D:\DA projects\Supply Chain Analysis\DataCoSupplyChainDataset.csv"

# Use a different encoding to avoid UnicodeDecodeError
data = pd.read_csv(file_path, encoding="ISO-8859-1")  # or encoding="latin1"

data.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fname                

In [67]:
# Treat NaN value before insert into MySQL:

data = data.where(pd.notna(data), None)  # Replace NaN with None (MySQL understands None as NULL)

In [69]:
myconnection = pymysql.connect(host = 'localhost', user = 'root', passwd = 'Root*#', database = 'Supply_Chain')

In [70]:
curr = myconnection.cursor()

In [71]:
# Check the number of columns in your DataFrame:
no_col = len(data.columns)

In [72]:
# Generate a string with the same number of placeholder as the number of columns
placeholder = ','.join(['%s'] * no_col)    # %s is the placeholder for all datatypes

In [73]:
curr.execute("CREATE TABLE sales ( \
    `Type` TEXT, \
    `Days for shipping (real)` TEXT, \
    `Days for shipment (scheduled)` TEXT, \
    `Benefit per order` TEXT, \
    `Sales per customer` TEXT, \
    `Delivery Status` TEXT, \
    `Late_delivery_risk` TEXT, \
    `Category Id` TEXT, \
    `Category Name` TEXT, \
    `Customer City` TEXT, \
    `Customer Country` TEXT, \
    `Customer Email` TEXT, \
    `Customer Fname` TEXT, \
    `Customer Id` VARCHAR(255), \
    `Customer Lname` TEXT, \
    `Customer Password` TEXT, \
    `Customer Segment` TEXT, \
    `Customer State` TEXT, \
    `Customer Street` TEXT, \
    `Customer Zipcode` TEXT, \
    `Department Id` TEXT, \
    `Department Name` TEXT, \
    `Latitude` DECIMAL(10,6), \
    `Longitude` DECIMAL(10,6), \
    `Market` TEXT, \
    `Order City` TEXT, \
    `Order Country` TEXT, \
    `Order Customer Id` TEXT, \
    `Order Date` TEXT, \
    `Order Id` TEXT, \
    `Order Item Cardprod Id` TEXT, \
    `Order Item Discount` DECIMAL(10,2), \
    `Order Item Discount Rate` DECIMAL(10,2), \
    `Order Item Id` TEXT, \
    `Order Item Product Price` DECIMAL(10,2), \
    `Order Item Profit Ratio` DECIMAL(10,2), \
    `Order Item Quantity` INT, \
    `Sales` DECIMAL(10,2), \
    `Order Item Total` DECIMAL(10,2), \
    `Order Profit Per Order` DECIMAL(10,2), \
    `Order Region` TEXT, \
    `Order State` TEXT, \
    `Order Status` TEXT, \
    `Order Zipcode` TEXT, \
    `Product Card Id` TEXT, \
    `Product Category Id` TEXT, \
    `Product Description` TEXT, \
    `Product Image` TEXT, \
    `Product Name` TEXT, \
    `Product Price` DECIMAL(10,2), \
    `Product Status` TEXT, \
    `Shipping Date` TEXT, \
    `Shipping Mode` TEXT \
)")

0

In [74]:
insert_values = "insert into sales values ({})".format(placeholder)

In [75]:
for i in range(len(data)):
    curr.execute(insert_values, tuple(data.iloc[i]))
    myconnection.commit()